# Resting State Processing - Neurogunther on BMAP

Numbering of steps corresponds to Kathy's scripts.

There are different design steps. The preprocessing FEAT (step 2b) does 6mm spatial smoothing 

In [1]:
source "$GUNTHERDIR/include/Asourceall.sh"

### Directories and File Names

In [2]:
rawdirs="RFMRI_REST_AP_7 RFMRI_REST_AP_8 RFMRI_REST_AP_9 RFMRI_REST_AP_10 RFMRI_REST_AP_11 RFMRI_REST_AP_12"
    # Possible names of the raw resting state data. Differences due to scanner collection order, ect.
basedir="/ifs/faculty/mirella/fmri/Prisma"

homedir="/ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis"

funcdir="restingHCP"
    # Directory where resting state analysis will be put in each subject directory
featdir_preprocess="${funcdir}/feats/ICAaroma_6mm_4Dmeanintensity.feat"

featdir_tissueregress="${funcdir}/feats/ICAaroma_global_tissueregress.feat"

options="global"

scriptdir="/ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/Scripts"


### Subjects

In [10]:
subjects_new="TD140_ETW"
#subjects_restingmeanfunc="TD140_ETW"

In [ ]:
start=$(date +%s.%N)

# HERE BE CODE

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

## 1. Preprocess and Register Data
### 1a. Bring Subject Data Over From Scanner--this is typically done by SRAs well in advance of analysis
### 1b. Check for raw resting data and copy into its only processing directory (funcdir)

In [15]:
#Look through two naming options and counts if there are multiple resting-state scans within the raw folder.
for subject in ${subjects_new}; do
    for raw in ${rawdirs}; do 
        counterVar=0
        for item in ${basedir}/subjects/${subject}/raw/${raw}; do
            counterVar=$(( ${counterVar}+1 ))
            #echo "${item}"
        done
        if (( ${counterVar}>1 )); then
            echo "Check ${subject} because ${counterVar} raw resting-state files exist."
            #echo ${basedir}/subjects/${subject}/raw/${raw} #Can echo the file names if you so wish
        fi

#If subject only has one raw resting-state scan, and there isn't already a raw in the named resting-state folder, then copy their raw scan to the subjects resting-state folder
        if [ ${counterVar} -eq 1 ]; then
            if [ -d ${basedir}/subjects/${subject}/raw/${raw} ]; then #Copy from raw folder only if it exists
                if [ ! -f ${basedir}/subjects/${subject}/${funcdir}/raw.nii.gz ]; then #Copy to new folder only if there's not already a raw in that folder
                    zinfo "${subject}: Copying raw.nii.gz from raw to new ${funcdir} folder"
                    mkdir ${basedir}/subjects/${subject}/${funcdir}
                    cp ${basedir}/subjects/${subject}/raw/${raw}/* ${basedir}/subjects/${subject}/${funcdir}/raw.nii.gz
                    chmod -R 777 ${basedir}/subjects/${subject}/${funcdir}/
                fi
            fi
        fi
    done
done

TD140_ETW: Copying raw.nii.gz from raw to new restingHCP folder


In [16]:
for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    #Get info of number of vols in raw scan
    subVolInfo=`fslinfo ${subdir}/raw.nii.gz`
    echo $subVolInfo
    subVolInfoNum=`echo ${subVolInfo} | awk '{print $10}'` #Sometimes this is $10, othertimes $12 seemingly depending on the computer or dataset
    echo $subVolInfoNum
    #Check raw scan length and act accordingly
    if [ ${subVolInfoNum} -eq 240 ]; then
        echo ${subject}': 8 minute scan, creating 6 minute version'
        cp ${basedir}/subjects/${subject}/${funcdir}/raw.nii.gz ${subdir}/raw_fullVolVersion.nii.gz
        fslroi ${subdir}/raw_fullVolVersion.nii.gz ${subdir}/raw.nii.gz 0 165	
    elif [ ${subVolInfoNum} -eq 165 ]; then
        echo ${subject}': 6 minute scan, ignoring subject'
    else
        echo ${subject}': Scan was neither 6 nor 8 minutes, was instead '${subVolInfoNum} 'volumes. Ignoring subject'
    fi

done

data_type UINT16 dim1 104 dim2 104 dim3 72 dim4 667 datatype 512 pixdim1 2.000000 pixdim2 2.000000 pixdim3 2.000000 pixdim4 0.720000 cal_max 0.0000 cal_min 0.0000 file_type NIFTI-1+
667
TD140_ETW: Scan was neither 6 nor 8 minutes, was instead 667 volumes. Ignoring subject


### 1c. Preprocessing 

In [20]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    if [ -f ${basedir}/subjects/${subject}/${funcdir}/func_task_pp.nii.gz ]; then
        echo "${subject}: Already had the func_task_pp.nii.gz file. Skipping step."
    else #Only run the preprocessing script if the func_task_pp file doesn't already exist (i.e. the script hasn't been run yet)
        zcmd "g_preprocessFunc_fslBET.sh -B ${basedir} -R ${funcdir} -S ${subject}"
    fi
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

X: g_preprocessFunc_fslBET.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/g_preprocessFunc_fslBET.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -S TD140_ETW'
reading list of subjects given by user
Running subjects: TD140_ETW


Running TD140_ETW
X: mkdir /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/1_preprocess 2> /dev/null; echo -n ''
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/1_preprocess
Not timeshifting
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/raw.nii.gz /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/1_preprocess/1_func_ts.nii.gz
Deobliquing
X: 3drefit -deoblique /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/1_preprocess/1_func_ts.nii.gz
++ 3drefit: AFNI version=AFNI_16.0.00 (Jan  1 2016) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/resti

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 7810 sec = 2.17 hours</span>

### 1c. Create mean functional image for registration (restingmeanfunc)

In [23]:
start=$(date +%s.%N)


for subject in ${subjects_new}; do
    if [ ! -f ${basedir}/subjects/${subject}/anat_hires/brain.nii.gz ]; then #Only run preprocessing script if it seems it hasn't been run before
        zcmd "cd ${basedir}/subjects/${subject}/restingHCP"
        zcmd "fslmaths raw.nii.gz -Tmean restingmeanfunc"
        zcmd "fslmaths restingmeanfunc.nii.gz restingmeanfuncshort.nii.gz -odt short"
        zcmd "chmod 777 restingmeanfuncshort.nii.gz"
        zcmd "cd ${basedir}/subjects/${subject}/"
        zcmd "mkdir -m 777 restingmeanfunc"
        zcmd "cp -r ${basedir}/subjects/${subject}/restingHCP/restingmeanfuncshort.nii.gz ${basedir}/subjects/${subject}/restingmeanfunc/"
        zcmd "mv ${basedir}/subjects/${subject}/restingmeanfunc/restingmeanfuncshort.nii.gz ${basedir}/subjects/${subject}/restingmeanfunc/raw.nii.gz"
        zcmd "g_preprocessAnat.sh -B ${basedir} -A restingmeanfunc -S ${subject}"
    fi
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP
X: fslmaths raw.nii.gz -Tmean restingmeanfunc
X: fslmaths restingmeanfunc.nii.gz restingmeanfuncshort.nii.gz -odt short
X: chmod 777 restingmeanfuncshort.nii.gz
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/
X: mkdir -m 777 restingmeanfunc
X: cp -r /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/restingmeanfuncshort.nii.gz /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc/
X: mv /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc/restingmeanfuncshort.nii.gz /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc/raw.nii.gz
X: g_preprocessAnat.sh -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/g_preprocessAnat.sh -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -S TD140_ETW'
reading list of subjects given by user
Running subjects: TD140_ETW


Running TD140_ETW
X: cd /ifs/facul

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 150 sec = 2.5 min</span>

### 1d. Register all volumes to Mean Resting State Image (restingmeanfunc)

In [25]:
start=$(date +%s.%N)

zcmd "g_regFunc.sh -B ${basedir} -A restingmeanfunc -R ${funcdir} -S ${subjects_new}"

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

X: g_regFunc.sh -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -R restingHCP -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/g_regFunc.sh -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -R restingHCP -S TD140_ETW'
reading list of subjects given by user
Running subjects: TD140_ETW


Running TD140_ETW
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP
X: mkdir /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/2_reg 2> /dev/null; echo -n ''
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/2_reg
Copying over files
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc/brain.nii.gz highres
X: fslmaths /usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz standard
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_example.nii.gz example_func
Register Func to T1
X: flirt -ref highres -in example_func -out example_func2highres_1 -omat example_func2highres_1.mat -

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 97 sec = 1.5 min</span>

## 2. Processing to prepare for ICA-AROMA: BET, Spatial Smoothing, 4D Mean Intensity Normalization

### 2a. Run BET to create a mask for ICA-AROMA

In [41]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    cd ${subdir}/
    zinfo "Running bet for ${subject}"
    bet func_example.nii.gz func_example_bet.nii.gz -f 0.3 -n -m -R
    chmod 777 *
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

Running bet for TD140_ETW
Runtime was 5.915501356124878


____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 6 sec</span>

### 2b. Run FEAT with 6mm spatial smoothing and 4D mean intensity normalization

In [43]:
for subject in ${subjects_new}; do
        subdir="${basedir}/subjects/${subject}/${funcdir}"
        featdir="${subdir}/feats"
        subVolInfo=`fslinfo ${subdir}/func_task_pp.nii.gz`
        oneSubVols=`echo $subVolInfo | awk '{print $10}'` #Sometimes this is $10, othertimes $12 seemingly depending on the computer or dataset. Check below which one it is for this list of subs.
            fslinfo ${subdir}/func_task_pp.nii.gz
            echo $subVolInfo | awk '{print $10}'
            echo $subVolInfo | awk '{print $12}'
            echo $oneSubVols

        ### Create feat for each subject
        ztitle "Creating Template Feat for ${oneSubVols} volumes"
        zcmd "sed 's/zzz/${oneSubVols}/g' ${homedir}/DesignFiles/ICAaroma_6mm_4Dmeanintensity_Template_Vols.fsf > ${homedir}/DesignFiles/ICAaroma_6mm_4Dmeanintensity_${oneSubVols}_Vols.fsf"

        ### Run feat for each subject
        ztitle "Running Preprocessing FEAT for ${subject} with 6mm spatial smoothing and 4D mean intensity normalization"
        zcmd "g_runFeats.sh -B ${basedir} -R ${funcdir} -T ${homedir}/DesignFiles/ICAaroma_6mm_4Dmeanintensity_${oneSubVols}_Vols.fsf -j xxxxxx -S ${subject}"
done


data_type      FLOAT32
dim1           104
dim2           104
dim3           72
dim4           667
datatype       16
pixdim1        2.000000
pixdim2        2.000000
pixdim3        2.000000
pixdim4        0.720000
cal_max        0.0000
cal_min        0.0000
file_type      NIFTI-1+
667
16
667
Creating Template Feat for 667 volumes
X: sed 's/zzz/667/g' /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_6mm_4Dmeanintensity_Template_Vols.fsf > /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_6mm_4Dmeanintensity_667_Vols.fsf
Running Preprocessing FEAT for TD140_ETW with 6mm spatial smoothing and 4D mean intensity normalization
X: g_runFeats.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -T /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_6mm_4Dmeanintensity_667_Vols.fsf -j xxxxxx -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/g_runFeats.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -T /ifs/fac

____
While it appears that the script finished in around 6 seconds, the FEAT is still running in the background for several hours (5.2 hrs for 1 subject on BMAP). See Log to know when finished:

##### On BMAP:

< /ifs/faculty/mirella/fmri/Prisma/subjects/***SUBJECT***/restingHCP/feats/ICAaroma_6mm_4Dmeanintensity.feat/report_log.html >

##### Using OSXFUSE or similar, 
copy/paste into browser on your computer:

< ***yourOSXFUSEmount***/subjects/***SUBJECT***/restingHCP/feats/ICAaroma_6mm_4Dmeanintensity.feat/report_log.html >

< /Users/emilywood/Desktop/BMAP/Prisma/subjects/***SUBJECT***/restingHCP/feats/ICAaroma_6mm_4Dmeanintensity.feat/report_log.html >

### 2c. Change permissions for all the FEATs you just ran

In [44]:
for subject in ${subjects_new}; do
    zinfo "Changing permissions for ${subject} for Preprocessing FEAT with 6mm spatial smoothing and 4D mean intensity normalization"
    cd ${basedir}/subjects/${subject}/${funcdir}/feats/ICAaroma_6mm_4Dmeanintensity.feat
    chmod -R 777 *
done

Changing permissions for TD140_ETW for Preprocessing FEAT with 6mm spatial smoothing and 4D mean intensity normalization


### 2d. Check FEATs to make sure run through without errors

In [45]:
for subject in ${subjects_new}; do
    /usr/local/anaconda2/bin/python2.7 /ifs/faculty/mirella/fmri/Pelphrey/RestingStateAnalysis/ACE_Girls/Scripts/checkFeatReports_FlexibleFunction.py ${subject} ${basedir} ${featdir_preprocess}
done

TD140_ETW: no errors


## 3. ICA-AROMA
### 3a. ICA AROMA

In [46]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    zinfo "ICA AROMA time for ${subject}"
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    featdir="${subdir}/feats"
    /usr/local/anaconda2/bin/python2.7 /ifs/faculty/mirella/fmri/ASD/RestingStateAnalysis/Project4_OlderCohort_Longitudinal/scripts/ICA_AROMA/ICA_AROMA.py -in ${featdir}/ICAaroma_6mm_4Dmeanintensity.feat/filtered_func_data.nii.gz -out ${subdir}/ICA_AROMA -mc ${subdir}/1_preprocess/mc/prefiltered_func_data_mcf.par -affmat ${subdir}/2_reg/example_func2standard.mat -m ${subdir}/func_example_bet_mask.nii.gz
    cd ${subdir}/
    chmod -R 777 *
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

ICA AROMA time for TD140_ETW

------------------------------- RUNNING ICA-AROMA ------------------------------- 
--------------- 'ICA-based Automatic Removal Of Motion Artifacts' --------------- 

Step 1) MELODIC
Step 2) Automatic classification of the components
  - registering the spatial maps to MNI
  - extracting the CSF & Edge fraction features
  - extracting the Maximum RP correlation feature
  - extracting the High-frequency content feature
  - classification
Step 3) Data denoising

----------------------------------- Finished -----------------------------------

Runtime was 44957.38413667679


____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 44957 sec = 12.5 hrs</span>

### 3b. Copy and rename the denoised functional file for further processing 

In [47]:
for subject in ${subjects_new}; do
    zinfo "Copying and renaming denoised func file for ${subject}"
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    icadir="${basedir}/subjects/${subject}/${funcdir}/ICA_AROMA/"
    cp ${icadir}/denoised_func_data_nonaggr.nii.gz ${subdir}/func_task_pp_ICAaroma.nii.gz
done

Copying and renaming denoised func file for TD140_ETW


## 4. Segment Brain
### 4a. Create functional mask for subject

In [48]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"		
    ztitle "Creating functional mask for ${subject}"
    zcmd "fslmaths ${subdir}/func_task_pp_ICAaroma -Tmax -bin ${subdir}/func_mask" 
    chmod 777 ${subdir}/func_mask.nii.gz
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

Creating functional mask for TD140_ETW
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma -Tmax -bin /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_mask
Runtime was 34.872947692871094


____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 35 sec </span>

### 4b. Segment brain and register to functional space

In [49]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    ztitle "Segmenting ${subject}'s brain and registering to functional space"
    zinfo "Segmenting brain for ${subject}"
    zcmd "gfc_segment.sh -u 2 -f -B ${basedir} -A restingmeanfunc -R ${funcdir} -S ${subject}"
    cd ${subdir}/
    chmod 777 *
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

Segmenting TD140_ETW's brain and registering to functional space
Segmenting brain for TD140_ETW
X: gfc_segment.sh -u 2 -f -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -R restingHCP -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/gfc_segment.sh -u 2 -f -B /ifs/faculty/mirella/fmri/Prisma -A restingmeanfunc -R restingHCP -S TD140_ETW'
reading list of subjects given by user
Running subjects: TD140_ETW


Running TD140_ETW
Segmenting the brain and generating probability maps of gray, white, and csf
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc
X: fast -t 2 -g -p -o segment /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingmeanfunc/brain.nii.gz
Setup to create masks for 3 nuisance signals
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP
X: mkdir /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/3_segment 2> /dev/null; echo -n ''
X: cd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/3_seg

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 51 sec</span>

## 5. Bandpass Filter

In [50]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    zinfo "Band Pass Temporal Filter preprocessed between 0.1Hz and .01 Hz for ${subject}"
    #Updated bptf command as of 9/2015 for newest version of FSL. 
    #MUST CHANGE BPTF IF TR CHANGES. There are conflicting formulas on the internet, but this appears to be the one we used for our own data: sigma[vol] = filter_width[secs]/(2*TR[secs]).
    zcmd "fslmaths ${subdir}/func_task_pp_ICAaroma.nii.gz -Tmean ${subdir}/func_task_pp_ICAaroma_mean.nii.gz"
    zcmd "fslmaths ${subdir}/func_task_pp_ICAaroma.nii.gz -bptf 25 2.5 -add ${subdir}/func_task_pp_ICAaroma_mean.nii.gz ${subdir}/func_task_pp_ICAaroma_bptf.nii.gz"
    cd ${basedir}/subjects/${subject}/${funcdir}/
    chmod 777 *
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

Band Pass Temporal Filter preprocessed between 0.1Hz and .01 Hz for TD140_ETW
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma.nii.gz -Tmean /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma_mean.nii.gz
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma.nii.gz -bptf 25 2.5 -add /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma_mean.nii.gz /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma_bptf.nii.gz
Runtime was 738.2928292751312


____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 738 sec = 12.3 min</span>

## 6. Create CSF, WM, Global Nuisance Regressors

In [51]:
start=$(date +%s.%N)

for subject in ${subjects_new}; do
    subdir="${basedir}/subjects/${subject}/${funcdir}"
    regdir="${subdir}/2_reg"
    featdir="${subdir}/feats"
    zinfo "Create nuisance regressors for WM, CSF and global signal ${subject}"
    zcmd "3dmaskave -mask ${subdir}/mask_csf.nii.gz -quiet ${subdir}/func_task_pp_ICAaroma_bptf.nii.gz > ${subdir}/csf_bptf.1D"
    zcmd "3dmaskave -mask ${subdir}/mask_wm.nii.gz -quiet ${subdir}/func_task_pp_ICAaroma_bptf.nii.gz > ${subdir}/wm_bptf.1D"
    zcmd "3dmaskave -mask ${subdir}/mask_global.nii.gz -quiet ${subdir}/func_task_pp_ICAaroma_bptf.nii.gz > ${subdir}/global_bptf.1D"
    cd ${subdir}/
    chmod 777 *
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

Create nuisance regressors for WM, CSF and global signal TD140_ETW
X: 3dmaskave -mask /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/mask_csf.nii.gz -quiet /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma_bptf.nii.gz > /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/csf_bptf.1D
++ 3dmaskave: AFNI version=AFNI_16.0.00 (Jan  1 2016) [64-bit]
+++ 169 voxels survive the mask
X: 3dmaskave -mask /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/mask_wm.nii.gz -quiet /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_ICAaroma_bptf.nii.gz > /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/wm_bptf.1D
++ 3dmaskave: AFNI version=AFNI_16.0.00 (Jan  1 2016) [64-bit]
+++ 3294 voxels survive the mask
X: 3dmaskave -mask /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/mask_global.nii.gz -quiet /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/func_task_pp_IC

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 50 sec</span>

## 7. Run FEAT to regress out tissue nuisance regressors
### 7a. Run FEAT to regress out tissue nuisance regressors (no smoothing)

In [52]:
start=$(date +%s.%N)

for option in ${options}; do
for subject in ${subjects_new}; do	
        subdir="${basedir}/subjects/${subject}/${funcdir}"
        featdir="${subdir}/feats"	
        subVolInfo=`fslinfo ${subdir}/func_task_pp_ICAaroma_bptf.nii.gz`
        oneSubVols=`echo $subVolInfo | awk '{print $10}'` #Sometimes this is $10, othertimes $12 seemingly depending on the computer or dataset. Check below which one it is for this list of subs.
            fslinfo ${subdir}/func_task_pp_bptf.nii.gz
            echo $subVolInfo | awk '{print $10}'
            echo $subVolInfo | awk '{print $12}'
            echo $oneSubVols

        ### Create feat for each subject 
        ztitle "Creating Template Feat for ${oneSubVols} volumes"
        zcmd "sed 's/zzz/${oneSubVols}/g' ${homedir}/DesignFiles/ICAaroma_${option}_tissueregress_Template_Vols.fsf > ${homedir}/DesignFiles/ICAaroma_${option}_tissueregress_${oneSubVols}_Vols.fsf"

        ### Run feat for each subject
        ztitle "Running FEAT for ${subject} for ${option} to regress nuisance regressors"
        zcmd "g_runFeats.sh -B ${basedir} -R ${funcdir} -T ${homedir}/DesignFiles/ICAaroma_${option}_tissueregress_${oneSubVols}_Vols.fsf -j xxxxxx -S ${subject}"
done
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"

ERROR: Could not open file
667
16
667
Creating Template Feat for 667 volumes
X: sed 's/zzz/667/g' /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_global_tissueregress_Template_Vols.fsf > /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_global_tissueregress_667_Vols.fsf
Running FEAT for TD140_ETW for global to regress nuisance regressors
X: g_runFeats.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -T /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_global_tissueregress_667_Vols.fsf -j xxxxxx -S TD140_ETW
Executing: '/ifs/faculty/mirella/neurogunther/g_runFeats.sh -B /ifs/faculty/mirella/fmri/Prisma -R restingHCP -T /ifs/faculty/mirella/fmri/Prisma/RestingStateAnalysis/DesignFiles/ICAaroma_global_tissueregress_667_Vols.fsf -j xxxxxx -S TD140_ETW'
reading list of subjects given by user
Running subjects: TD140_ETW


Running TD140_ETW
...skipping check of final output
X: mkdir -p /ifs/faculty/mirella/fmri/P

____
Even though it says you were done in 9 sec, the FEAT is running in the background.

<span style="font-family:courier;">Runtime for 1 subject on BMAP was 83 min</span>

### 7b. Change permissions for the FEATs you just ran

In [53]:
for option in ${options}; do
for subject in ${subjects_new}; do
    zinfo "Changing permissions for ${subject} for ${global} for tissue regression FEAT"
    chmod -R 777 ${basedir}/subjects/${subject}/${funcdir}/feats/ICAaroma_${option}_tissueregress.feat
done
done

Changing permissions for TD140_ETW for  for tissue regression FEAT


### 7c. Check FEATs

In [55]:
for subject in ${subjects_new}; do
    /usr/local/anaconda2/bin/python2.7 /ifs/faculty/mirella/fmri/Pelphrey/RestingStateAnalysis/ACE_Girls/Scripts/checkFeatReports_FlexibleFunction.py ${subject} ${basedir} ${featdir_tissueregress}
done

TD140_ETW: no errors


## Normalize and register residuals

In [60]:
start=$(date +%s.%N)

for option in ${options}; do
    for subject in ${subjects_new}; do
        subdir="${basedir}/subjects/${subject}/${funcdir}"
        regdir="${subdir}/2_reg"
        featdir="${subdir}/feats"

        ##### Normalize residuals
        ztitle "Normalize and register residuals for ${subject}"
        zinfo "Normalize residuals for ${subject}"
        zcmd "fslmaths ${basedir}/subjects/${subject}/${funcdir}/feats/ICAaroma_${option}_tissueregress.feat/stats/res4d.nii.gz -Tmean ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/mean_res4d_tmp"
        zcmd "fslmaths ${basedir}/subjects/${subject}/${funcdir}/feats/ICAaroma_${option}_tissueregress.feat/stats/res4d.nii.gz -Tstd ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/std_res4d_tmp"
        zcmd "fslmaths ${basedir}/subjects/${subject}/${funcdir}/feats/ICAaroma_${option}_tissueregress.feat/stats/res4d.nii.gz -sub ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/mean_res4d_tmp -div ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/std_res4d_tmp -add 100 -mas ${featdir}/ICAaroma_${option}_tissueregress.feat/mask.nii.gz ${featdir}/ICAaroma_${option}_tissueregress.feat/normres4d.nii.gz"
        zcmd "rm ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/*res4d_tmp*"


        ##### Register residuals and normalized residuals to standard space (assuming linear registration has been done)
        zinfo "Register residuals to standard space for ${subject}"
        zcmd "flirt -in ${featdir}/ICAaroma_${option}_tissueregress.feat/stats/res4d -ref ${regdir}/standard -out ${featdir}/ICAaroma_${option}_tissueregress.feat/res4d2standard -applyxfm -init ${regdir}/example_func2standard.mat"		

        zinfo "Register normalized residuals to standard space for ${subject}"
        zcmd "flirt -in ${featdir}/ICAaroma_${option}_tissueregress.feat/normres4d -ref ${regdir}/standard -out ${featdir}/ICAaroma_${option}_tissueregress.feat/normres4d2standard -applyxfm -init ${regdir}/example_func2standard.mat"

        ###Change permissions 
        zinfo "Changing permissions for ${subject} for whole folder"
        chmod -R 777 ${subdir}/*

    done
done

end=$(date +%s.%N)    
runtime=$(python -c "print(${end} - ${start})")

echo "Runtime was $runtime"


Normalize and register residuals for TD140_ETW
Normalize residuals for TD140_ETW
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/res4d.nii.gz -Tmean /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/mean_res4d_tmp
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/res4d.nii.gz -Tstd /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/std_res4d_tmp
X: fslmaths /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/res4d.nii.gz -sub /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/mean_res4d_tmp -div /ifs/faculty/mirella/fmri/Prisma/subjects/TD140_ETW/restingHCP/feats/ICAaroma_global_tissueregress.feat/stats/std_res

____
<span style="font-family:courier;">Runtime for 1 subject on BMAP was 587 sec = 9.8 min</span>